# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

## Question 0

In [22]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


In [4]:
df_youtube["tags"]=df_youtube["tags"].apply(lambda x : x.split("|")) # On transforme la colonne tags en liste de string pour utiliser au mieux les méthodes de mongo.
df_youtube["tags"]

0       [logan paul vlog, logan paul, logan, paul, oly...
1       [Apple, iPhone 10, iPhone Ten, iPhone, Portrai...
2                                                [[none]]
3       [apple iphone x hands on, Apple iPhone X, iPho...
4       [jacksfilms, parody, parodies, iphone, iphone ...
                              ...                        
7987    [sinkhole, how do sinkholes form, sinkhole in ...
7988    [Business Insider, Donald Trump, Hillary Clint...
7989    [LP, Death Valley, Other People, Lost On You, ...
7990    [YouTube FanFest, #YTFF, Washington DC, USA, Y...
7991    [LIGO, Virgo, collaboration, space, science, n...
Name: tags, Length: 7992, dtype: object

In [5]:
df_cat = pd.read_json("./data/US_category_id.json")
df_cat.head(2)

,kind,etag,items
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...","{'kind': 'youtube#videoCategory', 'etag': '""m2..."


In [6]:
# On crée une dataframe avec la colonne items afin de la nettoyer et de récupérer id.
df_items = pd.DataFrame(df_cat['items'].tolist())
df_items.head(2)

,kind,etag,id,snippet
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."


In [7]:
# On crée une dataframe avec la colonne snippets dans items afin de récupérer title.
df_snippet = pd.DataFrame(df_items['snippet'].tolist())
df_snippet.head(2)

,channelId,title,assignable
0,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True


In [8]:
# On ajoute title à la dataframe de items.
df_items["title"] = df_snippet["title"]
df_items.head(2)

,kind,etag,id,snippet,title
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit...",Film & Animation
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit...",Autos & Vehicles


In [9]:
# Et on supprime snippet puisque nous avons récupéré title.
df_items=df_items.drop("snippet", 1)
df_items.head(2)

,kind,etag,id,title
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,Film & Animation
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,Autos & Vehicles


In [10]:
# Nous n'avons besoin que de id et title pour fusionner le fichier US_category_id.json et USvideos.csv
df_items=df_items.drop("kind", 1)
df_items=df_items.drop("etag", 1)
df_items.head(2)

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles


In [11]:
df_items["id"]=df_items["id"].astype(int)
df_items.head(2)

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles


In [12]:
# On change le nom de title avant de fusionner puisqu'il y a déjà une colonne title dans la dataframe df.
df_items = df_items.rename(columns={'title': 'category'})

In [13]:
# On fusionne les deux fichiers.
df = pd.merge(df_youtube, df_items, how='left', left_on=['category_id'], right_on=['id'])
df.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,id,category
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,24,Entertainment
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,28,Science & Technology


In [14]:
# L'ensemble des données n'est pas forcément nécessaire donc nous avons passé en revue les questions de l'exercice pour supprimer les colonnes dont nous n'aurons pas besoin.
df=df.drop("category_id", 1)
df.head(2)

,video_id,title,channel_title,tags,views,likes,dislikes,comment_total,thumbnail_link,date,id,category
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,"[logan paul vlog, logan paul, logan, paul, oly...",4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,24,Entertainment
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,"[Apple, iPhone 10, iPhone Ten, iPhone, Portrai...",7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,28,Science & Technology


In [16]:
df.isna().sum()
# Il n'y a pas de NA donc on peut importer les données.

video_id          0
title             0
channel_title     0
tags              0
views             0
likes             0
dislikes          0
comment_total     0
thumbnail_link    0
date              0
id                0
category          0
dtype: int64

In [19]:
# On importe les données.
DOCUMENTS = df.to_dict('records')
len(DOCUMENTS)

7992

In [18]:
collection.insert_many(DOCUMENTS)

## Question 1  

In [65]:
# On récupère toutes les vidéos de la chaîne Apple.
cur = collection.find({"channel_title":"Apple"})
next(cur)
#pour les avoir toutes :
#for document in cur :
#    print('-----')
#    print(document)

{'_id': ObjectId('5fc22edd80728c32d6aac698'),
 'video_id': 'K4wEI5zhHB0',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'channel_title': 'Apple',
 'tags': ['Apple',
  'iPhone 10',
  'iPhone Ten',
  'iPhone',
  'Portrait Lighting',
  'A11 Bionic',
  'augmented reality',
  'emoji',
  'animoji',
  'Face ID',
  'Apple Pay',
  'camera',
  'smartphone'],
 'views': 7860119,
 'likes': 185853,
 'dislikes': 26679,
 'comment_total': 0,
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'date': 13.09,
 'id': 28,
 'category': 'Science & Technology'}

## Question 2

In [72]:
# On compte le nombre de catégories différentes.
len(collection.distinct('category'))

16

In [73]:
# Voici la liste des catégories.
collection.distinct('category')

['Autos & Vehicles',
 'Comedy',
 'Education',
 'Entertainment',
 'Film & Animation',
 'Gaming',
 'Howto & Style',
 'Music',
 'News & Politics',
 'Nonprofits & Activism',
 'People & Blogs',
 'Pets & Animals',
 'Science & Technology',
 'Shows',
 'Sports',
 'Travel & Events']

In [68]:
# On peut affiner la recherche :
cur = collection.aggregate([{"$group" : {"_id" : "$category", "charactereNumberByCategory" : {"$sum" : 1}}}])
list(cur)

[{'_id': 'Howto & Style', 'charactereNumberByCategory': 869},
 {'_id': 'People & Blogs', 'charactereNumberByCategory': 882},
 {'_id': 'Science & Technology', 'charactereNumberByCategory': 512},
 {'_id': 'Film & Animation', 'charactereNumberByCategory': 378},
 {'_id': 'Music', 'charactereNumberByCategory': 1250},
 {'_id': 'Travel & Events', 'charactereNumberByCategory': 48},
 {'_id': 'Sports', 'charactereNumberByCategory': 410},
 {'_id': 'Autos & Vehicles', 'charactereNumberByCategory': 116},
 {'_id': 'Gaming', 'charactereNumberByCategory': 82},
 {'_id': 'Education', 'charactereNumberByCategory': 334},
 {'_id': 'Pets & Animals', 'charactereNumberByCategory': 116},
 {'_id': 'Comedy', 'charactereNumberByCategory': 755},
 {'_id': 'Nonprofits & Activism', 'charactereNumberByCategory': 14},
 {'_id': 'News & Politics', 'charactereNumberByCategory': 623},
 {'_id': 'Shows', 'charactereNumberByCategory': 2},
 {'_id': 'Entertainment', 'charactereNumberByCategory': 1601}]

## Question 3

In [ ]:
# On a déjà découpé les tags en listes et mis à jour les tags de chacun des documents dans la question 0.

## Question 4

In [79]:
# Pour afficher les vidéos les plus vues, nous devons avoir un ordre de grandeur en tête concernant le distribution des vues.
cur = collection.aggregate([
   { "$group": {
      "_id": "NaN",
      "MaximumValue": { "$max": "$views" },
      "AvgValue": { "$avg": "$views" }
   }}
])
list(cur)
# La vidéo la plus vue a 41 500 672 vues et la moyenne des vues est d'environ 939 026 vues. Nous allons afficher les vidéos les plus vues (dans le premier quart), nous choisissons les vidéos dont le nombre de vues est au dessus de 21 219 850 vues.

[{'_id': 'NaN', 'MaximumValue': 41500672, 'AvgValue': 939026.9064064064}]

In [82]:
cur = collection.find( { "views": { "$gt": 21219850 }}, {"video_id":1})
next(cur)
#pour les avoir toutes :
#for document in cur :
#    print('-----')
#    print(document)

{'_id': ObjectId('5fc22edd80728c32d6aac748'), 'video_id': 'tt2k8PGm-TI'}

## Question 5

In [86]:
# On compte le nombre moyen de vues en fonction de la catégorie.
cur = collection.aggregate([{"$group" : {"_id" : "$category", "avgNumberByCategory" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 'Entertainment', 'avgNumberByCategory': 1154868.4397251718},
 {'_id': 'Howto & Style', 'avgNumberByCategory': 537665.1806674339},
 {'_id': 'Shows', 'avgNumberByCategory': 8492.5},
 {'_id': 'News & Politics', 'avgNumberByCategory': 540955.5569823436},
 {'_id': 'Comedy', 'avgNumberByCategory': 1240073.294039735},
 {'_id': 'Nonprofits & Activism', 'avgNumberByCategory': 1110334.2142857143},
 {'_id': 'Pets & Animals', 'avgNumberByCategory': 651404.3879310344},
 {'_id': 'Education', 'avgNumberByCategory': 547582.4700598803},
 {'_id': 'Gaming', 'avgNumberByCategory': 681081.8170731707},
 {'_id': 'Autos & Vehicles', 'avgNumberByCategory': 607693.8706896552},
 {'_id': 'Sports', 'avgNumberByCategory': 728434.3243902439},
 {'_id': 'Travel & Events', 'avgNumberByCategory': 464041.0833333333},
 {'_id': 'Science & Technology', 'avgNumberByCategory': 924730.548828125},
 {'_id': 'Film & Animation', 'avgNumberByCategory': 1039472.6560846561},
 {'_id': 'Music', 'avgNumberByCategory': 1176553.6

## Question 6 

In [87]:
# On récupère les chaines Youtube avec la plus grande moyenne de likes. Pour cela, on va regarder le nombre de likes moyen de chaque chaîne.
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "avgNumberByChannel" : {"$avg" : "$likes"}}}])
list(cur)[:3]

[{'_id': 'Working at Microsoft', 'avgNumberByChannel': 207.0},
 {'_id': 'Alyosha Tanetović', 'avgNumberByChannel': 75.66666666666667},
 {'_id': 'Tom Scott', 'avgNumberByChannel': 24219.947368421053}]

In [94]:
# Puis on regarde la moyenne globale.
cur = collection.aggregate([
   { "$group": {
      "_id": "NaN",
      "AvgValue": { "$avg": "$likes" }
   }}
])
list(cur)

[{'_id': 'NaN', 'AvgValue': 34491.92655155155}]

In [91]:
# Et on regarde la chaîne ayant la moyenne de likes la plus élevée
cur = collection.aggregate([{"$group" : {"_id" : "$channel_title", "numberAvgLikes" : {"$avg" : "$likes"}}}, {
        "$sort": { "numberAvgLikes": -1 }},
    {
        "$limit": 1 
    }])
list(cur) 
# comme dans la question 4, on cherche les vidéos dont le nombre de likes moyen est supérieur à 733 088 likes.

[{'_id': 'ZaynVEVO', 'numberAvgLikes': 1431683.0}]

In [96]:
# Et on a les chaines Youtube avec la plus grande moyenne de likes.
like = 733088
cur = collection.aggregate([
    {"$group" : {
        "_id" : "$channel_title", 
        "numberAvgLikes" : {"$avg" : "$likes"}
        }},
   {
      '$match': {
        'numberAvgLikes': {
          '$gt': like
        }
      }
    }])
list(cur)

[{'_id': 'ibighit', 'numberAvgLikes': 1371766.25},
 {'_id': 'melanie martinez', 'numberAvgLikes': 911871.5},
 {'_id': 'BETNetworks', 'numberAvgLikes': 769687.625},
 {'_id': 'jypentertainment', 'numberAvgLikes': 758826.6666666666},
 {'_id': 'ZaynVEVO', 'numberAvgLikes': 1431683.0}]